# ICIJ Offshore Leaks Database

In the following we want to standardize the management of information related to countries stored in `Address` nodes.

There are $374 955$ `Address` nodes in the dataset.

The following Cypher query outputs the number of different leaks:
```
MATCH (a:Address)
RETURN distinct a.sourceID
```
We see that there are $10$ different leaks: `["Bahamas Leaks", "Offshore Leaks", "Panama Papers", "Paradise Papers - Appleby", "Paradise Papers - Aruba corporate registry", "Paradise Papers - Bahamas corporate registry", "Paradise Papers - Barbados corporate registry", "Paradise Papers - Malta corporate registry", "Paradise Papers - Nevis corporate registry", "Paradise Papers - Samoa corporate registry"]`

In [1]:
from dtgraph import Neo4jGraph, Rule, Transformation
hostname = "localhost"
uri = f"bolt://{hostname}:7687"
graph = Neo4jGraph(uri, database="neo4j", username="", password="")

## Experiment 1: Single rule refactoring
We start with `Address` nodes with a `sourceID` of `Paradise Papers - Malta corporate registry`.

These nodes store country information in both `country_code` and `country` attributes.
### Transformation rule
We look at the execution time of the transformation rule:

In [2]:
panama_malta_address_to_global = Rule('''
MATCH (a:Address { sourceID: "Paradise Papers - Malta corporate registry" }) 
WHERE a.country_code IS NOT NULL
GENERATE
(x = (a):T_Address {
    source = a.sourceID
})-[():T_LOCATED]->(y = (a.country_code):T_Country {
    name = a.country
})
''')

In [12]:
#panama_address_to_global._compile(with_diagnose=False)
#print(panama_address_to_global._compiled)

In [4]:
my_transform = Transformation([panama_malta_address_to_global], with_diagnose = False) # no need for diagnose's specific functionalities for the VLDB exp.
rep, tt = 2, 0
for i in range(rep):
    print(f"Iteration {i=}:")
    tt += my_transform.apply_on(graph)
    my_transform.abort(keep_index = True) # Strange bug: indexes created with the Neo4j Python driver cannot be used in subsequent query plans
avg_time = tt/rep
print(f"Average execution time: {avg_time:.3f} ms.")

Iteration i=0:
Index: Added 0 index, completed after 0 ms.
Rule: Added 246866 labels, created 123433 nodes, set 493153 properties, created 123240 relationships, completed after 3291 ms.
Abort: Deleted 123433 nodes, deleted 123240 relationships, completed after 738 ms.
Iteration i=1:
Index: Added 0 index, completed after 0 ms.
Rule: Added 246866 labels, created 123433 nodes, set 493153 properties, created 123240 relationships, completed after 3071 ms.
Abort: Deleted 123433 nodes, deleted 123240 relationships, completed after 623 ms.
Average execution time: 3181.000 ms.


### Input query
We now look at the execution time and number of rows of the input query.

In [5]:
panama_malta_address_to_global_input_rule = '''
MATCH (a:Address { sourceID: "Paradise Papers - Malta corporate registry" }) 
WHERE a.country_code IS NOT NULL
RETURN a'''
nb_rows, exec_time = graph.query(panama_malta_address_to_global_input_rule)
print(f"Execution time of the input rule: {exec_time} ms.")
print(f"Size of intermediate data: {nb_rows} rows.")

Execution time of the input rule: 1005 ms.
Size of intermediate data: 123240 rows.


### Results

Present the statistics and give number of conflicting elements in the output.

In [6]:
_, summary = panama_malta_address_to_global.apply_on(graph)
# clear output data 
my_transform._graph = graph
my_transform.abort(keep_index = True)
num_nodes = summary.counters.nodes_created
num_rels = summary.counters.relationships_created

Rule: Added 246866 labels, created 123433 nodes, set 493153 properties, created 123240 relationships, completed after 2840 ms.
Abort: Deleted 123433 nodes, deleted 123240 relationships, completed after 630 ms.


In [7]:
print(f"Ratio between intermediate data and size of the output: {(num_nodes + num_rels) / nb_rows:.3f}")
print(f"Time to compute the intermediary results, per row: {exec_time / nb_rows:.3f} ms.")
print(f"Time to construct the output, per element: {avg_time / (num_nodes + num_rels):.3f} ms.")

Ratio between intermediate data and size of the output: 2.002
Time to compute the intermediary results, per row: 0.008 ms.
Time to construct the output, per element: 0.013 ms.


We now look at the percentage of conflicting elements in the output.

In [8]:
panama_malta_address_to_global_with_diagnose = Rule('''
MATCH (a:Address { sourceID: "Paradise Papers - Malta corporate registry" }) 
WHERE a.country_code IS NOT NULL
GENERATE 
(x = (a):T_Address {
    source = a.sourceID
})-[():T_LOCATED]->(y = (a.country_code):T_Country {
    name = a.country
})
''')
my_transform = Transformation([panama_malta_address_to_global_with_diagnose], with_diagnose = True) # We want to retrieve conflicting elements
my_transform.apply_on(graph)
nc, ec = my_transform.diagnose()
my_transform.abort(keep_index = True) # Strange bug: indexes created with the Neo4j Python driver cannot be used in subsequent query plans
print(f"Percentages of conflicting nodes and edges, respectively: {(nc * 100) / num_nodes:.1f}% and {(ec * 100) / num_rels:.1f}%.")

Index: Added 0 index, completed after 1 ms.
Rule: Added 246866 labels, created 123433 nodes, set 493153 properties, created 123240 relationships, completed after 3383 ms.
NodeConflicts: There are currently 0 nodes in the database which have a conflict.
EdgeConflicts: There are currently 0 edges in the database which have a conflict.
Abort: Deleted 123433 nodes, deleted 123240 relationships, completed after 705 ms.
Percentages of conflicting nodes and edges, respectively: 0.0% and 0.0%.


Indeed, the following openCypher query's output shows that the FD `country_code -> country` holds on `Address` nodes having the attribute `country_code`.

```
MATCH (a:Address { sourceID: "Paradise Papers - Malta corporate registry" }) 
WHERE a.country_code IS NOT NULL
WITH a.country_code AS country_code, COUNT(DISTINCT a.country) AS corresponding_country_names
WHERE corresponding_country_names > 1
RETURN country_code, corresponding_country_names
```

## Experiment 2: Whole refactoring
We now consider all the cases in a single transformation.

Address nodes with a `sourceID` different from `Paradise Papers - Malta corporate registry` store country information in both `country_codes` and `countries` attributes.

Actually, the leak `Paradise Papers - Malta corporate registry` were an exception using `country_code` and `country`.

### Transformation rule
We look at the execution time of the transformation consisting of the rules:

In [9]:
panama_malta_address_to_global = Rule('''
MATCH (a:Address { sourceID: "Paradise Papers - Malta corporate registry" }) 
WHERE a.country_code IS NOT NULL
GENERATE
(x = (a):T_Address {
    source = a.sourceID
})-[():T_LOCATED]->(y = (a.country_code):T_Country {
    name = a.country
})
''')
# Create the missing nodes
panama_malta_address_to_global_undefined_country = Rule('''
MATCH (a:Address { sourceID: "Paradise Papers - Malta corporate registry" }) 
WHERE a.country_code IS NULL
GENERATE
(x = (a):T_Address {
    source = a.sourceID
})
''')
# For all other leaks, the attributes are in plural form
other_address_to_global = Rule('''
MATCH (a:Address)
WHERE a.sourceID <> "Paradise Papers - Malta corporate registry"
  AND a.country_codes IS NOT NULL
GENERATE
(x = (a):T_Address {
    source = a.sourceID
})-[():T_LOCATED]->(y = (a.country_codes):T_Country {
    name = a.countries
})
''')
other_address_to_global_undefined_countries = Rule('''
MATCH (a:Address)
WHERE a.sourceID <> "Paradise Papers - Malta corporate registry"
  AND a.country_codes IS NULL
GENERATE
(x = (a):T_Address {
    source = a.sourceID
})
''')

In [10]:
#other_address_to_global._compile(with_diagnose=False)
#print(other_address_to_global._compiled)

In [11]:
my_transform = Transformation([
    panama_malta_address_to_global, 
    panama_malta_address_to_global_undefined_country,
    other_address_to_global,
    other_address_to_global_undefined_countries,
], with_diagnose = False) # no need for diagnose's specific functionalities for the VLDB exp.
rep, tt = 2, 0
for i in range(rep):
    print(f"Iteration {i=}:")
    tt += my_transform.apply_on(graph)
    my_transform.abort(keep_index = True) # Strange bug: indexes created with the Neo4j Python driver cannot be used in subsequent query plans
avg_time = tt/rep
print(f"Average execution time: {avg_time:.3f} ms.")

Iteration i=0:
Index: Added 0 index, completed after 1 ms.
Rule: Added 246866 labels, created 123433 nodes, set 493153 properties, created 123240 relationships, completed after 3064 ms.
Rule: Added 52 labels, created 26 nodes, set 52 properties, created 0 relationships, completed after 73 ms.
Rule: Added 500272 labels, created 250136 nodes, set 1000451 properties, created 250105 relationships, completed after 7822 ms.
Rule: Added 3168 labels, created 1584 nodes, set 3168 properties, created 0 relationships, completed after 224 ms.
Abort: Deleted 375179 nodes, deleted 373345 relationships, completed after 1933 ms.
Iteration i=1:
Index: Added 0 index, completed after 0 ms.
Rule: Added 246866 labels, created 123433 nodes, set 493153 properties, created 123240 relationships, completed after 2851 ms.
Rule: Added 52 labels, created 26 nodes, set 52 properties, created 0 relationships, completed after 95 ms.
Rule: Added 500272 labels, created 250136 nodes, set 1000451 properties, created 2501

### Input queries
We now look at the total execution time and total number of rows of the input queries.

In [15]:
nb_rows, exec_time = 0, 0
panama_malta_address_to_global_input_rule = '''
MATCH (a:Address { sourceID: "Paradise Papers - Malta corporate registry" }) 
WHERE a.country_code IS NOT NULL
RETURN a'''
r, e = graph.query(panama_malta_address_to_global_input_rule)
nb_rows += r
exec_time += e
panama_malta_address_to_global_undefined_country_input_rule = '''
MATCH (a:Address { sourceID: "Paradise Papers - Malta corporate registry" }) 
WHERE a.country_code IS NULL
RETURN a'''
r, e = graph.query(panama_malta_address_to_global_undefined_country_input_rule)
nb_rows += r
exec_time += e
other_address_to_global_input_rule = '''
MATCH (a:Address)
WHERE a.sourceID <> "Paradise Papers - Malta corporate registry"
  AND a.country_codes IS NOT NULL
RETURN a'''
r, e = graph.query(other_address_to_global_input_rule)
nb_rows += r
exec_time += e
other_address_to_global_undefined_countries_input_rule = '''
MATCH (a:Address)
WHERE a.sourceID <> "Paradise Papers - Malta corporate registry"
  AND a.country_codes IS NULL
RETURN a'''
r, e = graph.query(other_address_to_global_undefined_countries_input_rule)
nb_rows += r
exec_time += e
print(f"Total execution time of the input rule: {exec_time} ms.")
print(f"Total size of intermediate data: {nb_rows} rows.")

Total execution time of the input rule: 2821 ms.
Total size of intermediate data: 374955 rows.
